In [1]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import ticker
import matplotlib.dates as mdates
from datetime import datetime, time ,date ,timedelta
from dateutil.relativedelta import relativedelta
import seaborn as sns
import gc
pd.options.mode.chained_assignment = None

In [2]:
import warnings
warnings.filterwarnings("ignore")

In [24]:
today = date.today().strftime("%Y%m%d")

<h1> Functions </h1>

<h3> Fonctions générales </h3>

In [3]:
def floatise (df, list_columns_to_floatise):
    for i in list_columns_to_floatise:
        df[i] = df[i].apply(lambda x: float(x.replace(',','.')))
    return (df)

In [4]:
def intise (df, list_columns_to_floatise):
    for i in list_columns_to_floatise:
        df[i] = df[i].astype(int)
    return (df)

In [5]:
def flatten (table):
    if type(table.columns)==pd.MultiIndex:
        columns_to_look = [name_tmp for name_tmp in table.columns]

        columns_df = [ str(t[0])+'_'+str(t[1]) for t in columns_to_look]
        columns_df.insert(0,table.index.name)

        df = pd.DataFrame(columns = columns_df)

        index = 0
        for i in table.index:
            row = [table[r][i] for r in columns_to_look]
            row.insert(0,i)
            df.loc[index] = row
            index = index + 1
        return(df)
    else :
        table = pd.DataFrame(table)
        table.reset_index(level=0, inplace=True)
        return table

In [6]:
def flatten_soft(dataframe):
    res = pd.DataFrame()
    res[dataframe.index.name] = dataframe.index
    for col in dataframe.columns:
        name_tmp=""
        for i in range(len(dataframe.columns[0])):
            name_tmp = name_tmp +'_'+str(col[i])
        res[str(name_tmp)] = dataframe[col].values
    return res

<h3> Fonctions particulières </h3>

In [7]:
def cohort_attribution(x):
    if x <= 2016:
        return 2016
    else :
        return x

In [8]:
def customer_category_attribution(x):
    if x==1:
        return 'One-timer'
    if x==2:
        return 'Two-timer'
    else:
        return 'Recurring'

In [9]:
dict_department = {'B':'BEBE','N':'PAP FILLE','U':'PAP GARCON','D':'CHAUSSURE','F':'FEMME',
'T':'TEXTILE HOME','P':'PUERICULTURE','W':'NON PAP ENFANT','K':'DECO','M':'CHAMBRE ET LITERIE','R':'JOUETS'}

In [10]:
repertoire = "C:/Users/pierrick/Documents/Vertbaudet/Germany/Source_file"


In [11]:
output_rep = "C:/Users/pierrick/Documents/Vertbaudet/Germany/Output_python"

<h1> Dataset creation </h1>

<h3> Import de la data </h3>

In [19]:
#columns = ['AUFTRNR', 'KDNR', 'ARTNRERF', 'WTR', 'ARTGRERF', 'DATERF', 'PREISERF', 'PREIS', 'EKP', 'PREISNET',
#           'RABATT', 'RABATTSATZ', 'MENGE', 'MENGERET', 'DEPARTEMENT', 'CAUFTRAGSPOS', 'MWST', 'MWSTRABATT', 
#           'ANZKINDER', 'ANLAGEDAT', 'DATAUFTRAG1', 'STRASSE', 'PLZ', 'ORT', 'BUNDESLAND', 'CLAND', 'CWERBESP', 
#           'DATWERBESP', 'CLIEFERSP', 'DATLIEFERSP', 'CADRESSSP', 'DATADRESSSP', 'UMSATZ1', 'DATLETZTAUFTR', 
#           'UMSLETZTAUFTR', 'ANZKATLETZTAUFTR', 'WKOSTLETZTAUFTR', 'DBKDNR', 'WKOSTENKUM', 'UMSATZKUM', 'UMSATZKUMTEL', 
#           'UMSATZKUMFAX', 'UMSATZKUMWEB', 'UMSATZLIEFKUM', 'RUECKSTAUFTR', 'ANZAUFTR', 'REFPRES', 'REFSTK', 'CODDOC', 
#           'PLANEKP', 'RAYON', 'FAMILLE', 'DEPART', 'CODMARQ', 'ARTGROESSE', 'DATAUFTRAG', 'DATERFASSUNG', 'DATVERARB', 
#           'GUTSCHEINWERT']

In [12]:
columns = ['AUFTRNR', 'KDNR', 'DATERF', 'PREIS', 'PREISNET', 'RABATT', 'RABATTSATZ', 'MENGE', 'MENGERET', 'DEPARTEMENT', 'CAUFTRAGSPOS', 'DATAUFTRAG1', 'PLZ']

# Import

In [13]:
net_demand = True

In [14]:
df = pd.DataFrame()
for file in os.listdir(repertoire):
    file_name_tmp = os.path.join(repertoire,file)
    df_tmp = pd.read_csv(file_name_tmp,sep=",",encoding= "unicode_escape", usecols=columns)
    
    df_tmp = df_tmp.loc[df_tmp.CAUFTRAGSPOS!='$null$']
    df_tmp.CAUFTRAGSPOS = df_tmp.CAUFTRAGSPOS.astype(int)
    df_tmp = df_tmp.loc[df_tmp.CAUFTRAGSPOS <= 7]
    
    ################### CONVERSION DES CHAMPS ###################
    df_tmp['DATERF'] = pd.to_datetime(df_tmp.DATERF)
    df_tmp['DATAUFTRAG1'] = pd.to_datetime(df_tmp.DATAUFTRAG1)
    df_tmp.DEPARTEMENT = df_tmp.DEPARTEMENT.map(dict_department) 
    df_tmp['PLZ'] = df_tmp['PLZ'].astype(str)
    
    ################### CREATION DES CHAMPS ###################
    df_tmp['TOT_SALES'] = (df_tmp.PREIS - df_tmp.RABATT)*(df_tmp.MENGE - df_tmp.MENGERET)
    df_tmp['NET_DEMAND'] = (df_tmp.PREIS - df_tmp.RABATT) * df_tmp.MENGE
    df_tmp['GROSS_DEMAND'] = df_tmp.PREIS * df_tmp.MENGE
    df_tmp['YEAR_FIRST_ORDER'] = df_tmp.DATAUFTRAG1.dt.year
    df_tmp['YEAR_ORDER'] = df_tmp.DATERF.dt.year
    df_tmp['COHORT'] = df_tmp.YEAR_FIRST_ORDER.apply(lambda x: cohort_attribution(x))
    df_tmp['MONTH_RECRUITMENT'] = df_tmp.DATAUFTRAG1.dt.month
    df_tmp['ID_ORDER'] = df_tmp.AUFTRNR.astype(str) + '_'+ df_tmp.KDNR.astype(str)
    
    if net_demand:
        cat_order = df_tmp.sort_values(by='NET_DEMAND',ascending=False).drop_duplicates(subset='ID_ORDER', keep='first',
                                                                                        inplace=False)
    else:
        cat_order = df_tmp.sort_values(by='TOT_SALES',ascending=False).drop_duplicates(subset='ID_ORDER', keep='first',
                                                                                        inplace=False)
    cat_order = cat_order [['ID_ORDER','DEPARTEMENT']]
    cat_order.columns = ['ID_ORDER','DEPARTEMENT_CMD']
    df_tmp = pd.merge(df_tmp,cat_order,on='ID_ORDER',how='left',suffixes=(False,False))
    
    df_tmp = df_tmp[['DATERF','DATAUFTRAG1','DEPARTEMENT','TOT_SALES','NET_DEMAND','GROSS_DEMAND',
                    'YEAR_FIRST_ORDER','YEAR_ORDER','COHORT','MONTH_RECRUITMENT','ID_ORDER','KDNR',
                     'DEPARTEMENT_CMD','RABATT','RABATTSATZ','PREIS', 'PLZ', ]]
    df = pd.concat([df,df_tmp])
    del(df_tmp)

################### CREATION DE CHAMPS ###################
################### UNIVERS RECRUTEMENT
cat_rec = df.sort_values(by='DATERF',ascending=True).drop_duplicates(subset='KDNR', keep='first', inplace=False)[['KDNR','DEPARTEMENT_CMD']] 
cat_rec.columns = ['KDNR','DEPARTEMENT_FIRST']
df = pd.merge(df,cat_rec,on='KDNR',how='left',suffixes=(False,False))

In [15]:
nb_commande = flatten(df.groupby(['KDNR']).agg({'ID_ORDER':pd.Series.nunique}))
nb_commande.columns = ['KDNR','NB_ORDERS']
df = pd.merge(df,nb_commande,on='KDNR',how='left',suffixes=(False,False))
df['CLIENT_CATEGORY'] = df.NB_ORDERS.apply(lambda x: customer_category_attribution(x))

In [17]:
#file_name = './data_map_GER.csv'
#df.to_csv(file_name,sep=";")

# Rev and nb client by Lander

In [17]:
# Add Bundesland column by merging with ZIP/Lander correspondance table

df_zip_lander = pd.read_csv('C:/Users/pierrick/Documents/Vertbaudet/Germany/German-Zip-Codes.csv', sep = ';').drop_duplicates(subset = ['Plz'], keep = 'first')
df_zip_lander['Plz'] = df_zip_lander['Plz'].astype(str)
df_zip_lander['Bundesland'] = df_zip_lander['Bundesland'].replace('Schlewig-Holstein', 'Schleswig-Holstein')

df = df.merge(df_zip_lander[['Plz', 'Bundesland']], how = 'left', left_on = 'PLZ', right_on = 'Plz')

In [18]:

df = df.merge(df_zip_lander[['Plz', 'Bundesland']], how = 'left', left_on = 'PLZ', right_on = 'Plz')

In [ ]:
#df.groupby('Bundesland').agg({'REVNET': 'sum', 'Bundesland':'first'}).to_csv('revnet_by_lander_'+str(year)+'.csv')
#df.groupby('Bundesland').agg({'KDNR':pd.Series.nunique, 'Bundesland':'first'}).to_csv('client_by_lander_'+str(year)+'.csv')

# Unbias with national statistics

### Load number of household by Lander

In [52]:
df_destatis = pd.read_csv('german_lander_nb_foyer_children.csv', encoding = 'unicode_escape', sep = ';')
df_destatis = df_destatis.groupby('LANDER').agg({'COUNT_2017':'sum', 'COUNT_2018':'sum', 'COUNT_2019':'sum'})

In [56]:
df = df.merge(df_destatis, how = 'left', left_on = 'Bundesland', right_on = 'LANDER')

### From nb client and rev by Lander to map csv

In [64]:
df_output = df.groupby(["YEAR_ORDER", "Bundesland"]).agg({"NET_DEMAND": sum, "KDNR": pd.Series.nunique, 'COUNT_2017': "first", 'COUNT_2018': "first", 'COUNT_2019': "first", 'YEAR_ORDER':'first'})

df_output["COUNT_YEAR"] = df_output["COUNT_2019"]
df_output[df_output["YEAR_ORDER"] == 2017]["COUNT_YEAR"] = df_output[df_output["YEAR_ORDER"] == 2017]["COUNT_2017"]
df_output[df_output["YEAR_ORDER"] == 2018]["COUNT_YEAR"] = df_output[df_output["YEAR_ORDER"] == 2018]["COUNT_2017"]

df_output['PENETRATION'] = df_output['KDNR'] / df_output['COUNT_YEAR']
df_output['REV_BY_CLIENT'] = df_output['NET_DEMAND'] / df_output['KDNR']
df_output['REV_BY_HOUSEHOLD'] = df_output['NET_DEMAND'] / df_output['COUNT_YEAR']

for year in range(2017, 2022):
    print('Mean penetration in Germany in '+str(year)+' :')
    print(df_output[df_output["YEAR_ORDER"] == year]['KDNR'].sum()/df_output[df_output["YEAR_ORDER"] == year]['COUNT_YEAR'].sum())
    
    print('Mean rev by client in Germany in '+str(year)+' :')
    print(df_output[df_output["YEAR_ORDER"] == year]['NET_DEMAND'].sum()/df_output[df_output["YEAR_ORDER"] == year]['KDNR'].sum())


df_output.to_excel('GER_map_v3.xlsx')

Mean penetration in Germany in 2017 :
0.02765952250106857
Mean rev by client in Germany in 2017 :
173.22215910288548
Mean penetration in Germany in 2018 :
0.0267252854613177
Mean rev by client in Germany in 2018 :
156.43932713465605
Mean penetration in Germany in 2019 :
0.025283812664102095
Mean rev by client in Germany in 2019 :
139.20744199749478
Mean penetration in Germany in 2020 :
0.02949056603773585
Mean rev by client in Germany in 2020 :
139.86840697222507
Mean penetration in Germany in 2021 :
0.03121829394883068
Mean rev by client in Germany in 2021 :
139.26079403577538


In [42]:
gc.collect()

172

In [35]:
df_output.to_excel('GER_map_v.xlsx')

### Map category by Lander

In [19]:
df_output2 = df[(df['YEAR_ORDER']>=2017)&(df['COHORT'].isin(range(2017,2021)))].drop_duplicates('KDNR').groupby("Bundesland").agg({"CLIENT_CATEGORY": "value_counts"}).unstack().unstack()

In [20]:
df_output2 = df_output2.swaplevel().unstack()

In [ ]:
df_output2.sum().sum()

In [21]:
for cat in ['One-timer', 'Two-timer', 'Recurring']:
    print('% of '+cat+':')
    print(df_output2[cat].sum()/(df_output2.sum().sum()))

% of One-timer:
0.561216894130732
% of Two-timer:
0.19665552517552193
% of Recurring:
0.2421275806937461


In [22]:
div = df_output2.sum(axis = 1)
for cat in ['One-timer', 'Two-timer', 'Recurring']:
    df_output2[cat] = (df_output2[cat]/div)*100

In [23]:
df_output2

CLIENT_CATEGORY                         One-timer  Recurring  Two-timer
                Bundesland                                             
CLIENT_CATEGORY Baden-Württemberg       56.044762  24.423148  19.532090
                Bayern                  55.755368  24.469165  19.775467
                Berlin                  56.628926  23.805920  19.565154
                Brandenburg             55.016590  25.332959  19.650451
                Bremen                  58.432517  22.366944  19.200539
                Hamburg                 55.248424  24.751576  20.000000
                Hessen                  56.303380  23.975785  19.720835
                Mecklenburg-Vorpommern  55.100425  25.750486  19.149089
                Niedersachsen           56.438239  23.696638  19.865123
                Nordrhein-Westfalen     56.297931  24.131724  19.570345
                Rheinland-Pfalz         56.021627  24.187511  19.790862
                Saarland                57.512019  22.971755  19.516226
                Sachsen                 56.207538  24.127984  19.664478
                Sachsen-Anhalt          54.845738  25.426945  19.727317
                Schleswig-Holstein      57.631712  22.304901  20.063387
                Thüringen               56.129907  24.655193  19.214900

In [21]:
for year in range(2017,2022):
    div = df_output2['CLIENT_CATEGORY', year].sum(axis = 1)
    for cat in ['One-timer', 'Two-timer', 'Recurring']:
        df_output2['CLIENT_CATEGORY', year, cat] /= div
        df_output2['CLIENT_CATEGORY', year, cat] *= 100

In [57]:
df_output2.to_excel('./Vertbaudet/Germany/'+today+'_GER_categories_lander_map_cohor.xlsx')

# Average first year of order by Lander

In [35]:
df['SENIORITY'] = (pd.to_datetime(df['DATERF'])-pd.to_datetime(df['DATAUFTRAG1'])).dt.days

In [36]:
df['SENIORITY_y'] = df['SENIORITY']/365

In [37]:
df['SENIORITY_y'].head()

0    0.306849
1    0.306849
2    4.287671
3    4.287671
4    4.287671
Name: SENIORITY_y, dtype: float64

In [38]:
df_old = df[(df['SENIORITY_y'].notna())&(df['YEAR_ORDER']==2020)]

In [39]:
df_old = df[(df['SENIORITY_y'].notna())&(df['YEAR_ORDER']==2020)]
df_old = df[df["SENIORITY_y"]<20].sort_values(by='DATERF', ascending = True).drop_duplicates('KDNR', keep = 'last').groupby('Bundesland').agg({'SENIORITY_y': 'mean'})

In [40]:
df_old['SENIORITY_y'].mean()

2.4126627537008467

In [41]:
df_old.to_excel('./Vertbaudet/Germany/seniority_by_lander_v4.xlsx')